In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

In [ ]:
### Mape表をCSVとして保存する関数
### 引数はbenchmark:ベンチマーク名, FixClass:固定するベンチマーククラス, FixProcess:固定するプロセス数, DirPath:保存するディレクトリ
def SaveMapeTables(FixClass="B", FixProcess=256, DirPath = "./tmp_GenerateResources/"):
    for benchmark in benchmarks:
        MapeTableFixedClass = return_MapeTableFixedClass(benchmark, FixClass="B")
        MapeTableFixedProcess = return_MapeTableFixedProcess(benchmark, FixProcess=256)
        FixedClassFileName = f"MapeTableFixedClass_{benchmark}{FixClass}.csv"
        FixedProcessFileName = f"MapeTableFixedProcess_{benchmark}{FixProcess}.csv"
    #     print(f"FixedClassFileName={FixedClassFileName}, MapeTableFixedProcess={FixedProcessFileName}")
        if(type(MapeTableFixedClass) is pd.core.frame.DataFrame):
            print(f"FixedClassFileName={FixedClassFileName}")
            MapeTableFixedClass.to_csv(f"{DirPath}{FixedClassFileName}")
        if(type(MapeTableFixedProcess) is pd.core.frame.DataFrame):
            print(f"FixedProcessFileName={FixedProcessFileName}")
            MapeTableFixedProcess.to_csv(f"{DirPath}{FixedProcessFileName}")


SaveMapeTables(FixClass="C", FixProcess=256)

In [ ]:
# ##### 各関数での実行時間を保存するためのCSVの空データを作成するための処理 #####
# Processes = []
# for i in range(1,16):
#     Processes.append(i*i)
# for j in range(9):
#     Processes.append(2**j)
# Processes = list(set(Processes))
# Processes.sort()
# index = classes
# EmptyDF = pd.DataFrame(index=index, columns=Processes)
# for benchmark in benchmarks:
#     EmptyDF.to_csv(f"./csv_files/ExecTime@{benchmark}.csv")

In [ ]:
### クラス：TimeData
### 下記の関数(= return_TimeDataList())のために作成された
### メソッドreturn_AllData()で値のすべてを辞書形式で受けてることができる
class TimeData:
    def __init__(self, benchmark="cg", process="32", BenchmarkClass="C", time=-1):
        self.benchmark = benchmark
        self.process = process
        self.BenchmarkClass = BenchmarkClass
        self.time = time
    def return_AllData(self):
        return {"benchmark":self.benchmark, "process":self.process, "BenchmarkClass":self.BenchmarkClass, "time":self.time}

### 返値に独自クラスTimeDataのリストを返す
### 引数に実行プロセス数を取る
### 返値のリストの要素は引数のリストのプロセス数で実行されたベンチマークの
### ベンチマーク名・実行プロセス数・ベンチマーククラス・実行時間が記録されたTimeDataクラスのインスタンス
def return_TimeDataList(process = 256):
    return_list = []
    with open(f"./toGetProfile/toGetTime/TimeWith{process}.txt") as f:
        line_count = 0
        for line in f:
            line_count += 1
            if(line_count%3 == 1):
                benchmark = line[1:3].lower()
                # print(f"benchmark={benchmark}, len(benchmark)={len(benchmark)}")
                Data = TimeData(benchmark=benchmark, process=process)
            if(line_count%3 == 2):
                BenchmarkClass = line[-2]
                # print(f"BenchmarkClass={BenchmarkClass}, len(BenchmarkClass)={len(BenchmarkClass)}")
                Data.BenchmarkClass = BenchmarkClass
            if(line_count%3 == 0):
                Time = line[-25:]
                Time = Time.strip()
                # print(f"Time={Time}")
                Data.time = Time
                return_list.append(Data)
    return(return_list)


In [ ]:

### 関数:FillCSV()
### 引数は,benchmark:ベンチマーク名, process:実行プロセス数, BenchmarkClass:ベンチマーククラス, time:実行時間
### 引数として渡された値を適切なCSVに保存する
def FillCSV(benchmark = "cg",process = 256,BenchmarkClass = "A",time = "0.04"):
    time=float(time)
    CSVFilename = f"./csv_files/ExecTime@{benchmark}.csv"
    DataFrame = pd.read_csv(CSVFilename, index_col=0)
    DataFrame.at[BenchmarkClass, process] = time
    DataFrame.to_csv(CSVFilename)

In [ ]:
# Benchmarks = []
# for benchmark in benchmarks:
#     Benchmarks.append(benchmark.upper())
# print(Benchmarks)

Processes = [8, 16, 32, 64, 128, 256]
for process in Processes:
    TimeDataList = return_TimeDataList(process)
    for DictData in TimeDataList:
        benchmark = DictData.benchmark
        process = DictData.process
        BenchmarkClass = DictData.BenchmarkClass
        time = DictData.time
        FillCSV(benchmark=benchmark, process=process, BenchmarkClass=BenchmarkClass, time=time)

# print(TimeDataList[0].return_AllData())


In [ ]:
# ipynb形式のライブラリのインポート
%run ./lib.ipynb

### 誤差率の表を作成する関数
### 引数は,benchmark:関数名, predict_class:予測したいクラス, predict_process:予測したいプロセス数, FixProcess:固定する実行プロセス数, FixClass:固定するベンチマーククラス
### 返り値はなし
def GenerateErrorRateTable(benchmark = "cg",predict_class = "D",predict_process = 256,FixProcess = 64,FixClass = "B"):
    try:
        ErrorRateFixedProcessDF = return_ErrorRateFixedProcessDF(benchmark=benchmark, FixProcess=FixProcess, predict_class=predict_class)
    except:
        print("実行プロセスを固定した際の誤差率の表を取得するのに失敗したので、CSVとして保存できませんでした。")
    ErrorRateFixedProcessDF.to_csv(f"./tmp_GenerateResources/ErrorRateTableFixedProcess_{benchmark}{FixProcess}.csv")
    try:
        ErrorRateFixedClassDF = return_ErrorRateFixedClassDF(benchmark=benchmark, FixClass=FixClass, predict_process=predict_process)
    except:
        print("ベンチマーククラスを固定した際の誤差率の表を取得するのに失敗したので、CSVとして保存できませんでした。")
    ErrorRateFixedClassDF.to_csv(f"./tmp_GenerateResources/ErrorRateTableFixedClass_{benchmark}{FixClass}.csv")

### 誤差率のデータフレームを返す関数
### return_ErrorRateFixedProcessDF(), return_ErrorRateFixedClassDF()
def return_ErrorRateFixedProcessDF(benchmark="cg", FixProcess=64, predict_class="D"):
    FixProcessDF = return_FixedProcessModelDF(benchmark=benchmark, FixProcess=256)
    # print(FixProcessDF)
    for index in FixProcessDF.index.tolist():
        for column in FixProcessDF.columns.tolist():
            PredictNum = return_Predicted(FixProcessDF.at[index, column], ConvertBencharkClass_inNPB(predict_class))
            FixProcessDF.at[index, column] = return_ErrorRate(FunctionName=index, PredictNum=PredictNum, BenchmarkName=benchmark, BenchmarkClass=predict_class, Process=FixProcess)
    return (FixProcessDF)
def return_ErrorRateFixedClassDF(benchmark="cg", FixClass="B", predict_process=256):
    FixClassDF = return_FixedClassModelDF(benchmark, FixClass=FixClass)
    # print(FixClassDF)
    for index in FixClassDF.index.tolist():
        for column in FixClassDF.columns.tolist():
            PredictNum = return_Predicted(FixClassDF.at[index, column], predict_process)
            FixClassDF.at[index, column] = return_ErrorRate(FunctionName=index, PredictNum=PredictNum, BenchmarkName=benchmark, BenchmarkClass=FixClass, Process=predict_process)
    return(FixClassDF)

for benchmark in benchmarks:
    GenerateErrorRateTable(benchmark=benchmark)